In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/esoh/"
oCV_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

from scipy.io import savemat


In [2]:
## Added from old vrsion 


In [3]:
parameter_values = get_parameter_values()

In [4]:
parameter_values.search("Negative electrode reference exchange-current density [A.m-2(m3.mol)1.5]")

Negative electrode reference exchange-current density [A.m-2(m3.mol)1.5]	4.244e-06


In [5]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
# spm.print_parameter_info()
param=spm.param

In [6]:


cell = 4




In [7]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [8]:
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

drive_cycle = pd.read_csv(r"F:\Stefanapolu application\CRC\Drive Cycle\US06_default_5Ah.csv", comment="#", header=None).to_numpy()
drive_cycle2 = pd.read_csv(r"F:\Stefanapolu application\CRC\Drive Cycle\US06_GM_5Ah.csv", comment="#", header=None).to_numpy()

experiment = pybamm.Experiment( # defult D.C. DRC-DRC *1.25
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1.9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*520,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment2 = pybamm.Experiment( #lab cycling
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *300,
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

## GM_DCRDCR
experiment3 = pybamm.Experiment( #GM_ DRC-DRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment4 = pybamm.Experiment( #Gm-RCRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment5 = pybamm.Experiment( # defult D.C. RC-RC
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.8 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.7 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)



experiment6 = pybamm.Experiment( # defult D.C. DRC-DRC *1.25 with power
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         #"Discharge at C/4 for 1 hours", # should be P/4 
         "Discharge at 5.25 W for 1 hours", # ~~~ 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .75 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         #"Discharge at C/4 for 1 hours", # should be P/4 
         "Discharge at 5.25 W for 1 hours", # ~~~
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1.75 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*520,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

In [9]:
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]":1.4493e-07,   #2.5112e-07, #Using Sravan's new fit Dec27 2022
        "Negative electrode LAM constant proportional term [s-1]":1.3608e-06, #2.4741e-06,
        "Positive electrode LAM constant exponential term":1.199, #1.1942,
        "Negative electrode LAM constant exponential term":1.199,# 1.1942,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
        "Lithium plating kinetic rate constant [m.s-1]": 1.5778e-09,#Sravan fit dec 2022

    },
    check_already_exists=False,
)


In [10]:
# all_sumvars_dict1 = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [11]:
sim_long = pybamm.Simulation(spm, experiment=experiment2, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=SOC_0, save_at_cycles=1  )

2023-01-02 23:28:51.339 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/300 (19.586 ms elapsed) --------------------
2023-01-02 23:28:51.340 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:28:51.490 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 2/4: Rest for 5 min
2023-01-02 23:28:51.569 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:28:51.710 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:52.149 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.849 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:52.149 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/300 (830.175 ms elapsed) --------------------
2023-01-02 23:28:52.150 - [NOTICE] callbacks.on_step_start(182): Cycle 2/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:28:52.207 - [NOTICE] callbacks.on_step_start(182): Cyc

2023-01-02 23:28:54.418 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.817 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:54.419 - [NOTICE] callbacks.on_cycle_start(174): Cycle 13/300 (3.100 s elapsed) --------------------
2023-01-02 23:28:54.420 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:28:54.473 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 2/4: Rest for 5 min
2023-01-02 23:28:54.485 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:28:54.532 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:54.637 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.814 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:54.639 - [NOTICE] callbacks.on_cycle_start(174): Cycle 14/300 (3.319 s elapsed) --------------------
2023-01-02 23:28:54.640 - [NOTICE] callbacks.on_

2023-01-02 23:28:56.867 - [NOTICE] callbacks.on_step_start(182): Cycle 24/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:56.966 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.780 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:56.967 - [NOTICE] callbacks.on_cycle_start(174): Cycle 25/300 (5.648 s elapsed) --------------------
2023-01-02 23:28:56.969 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:28:57.035 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 2/4: Rest for 5 min
2023-01-02 23:28:57.047 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:28:57.097 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:57.192 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.777 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:57.192 - [NOTICE] callbacks.on_cycle

2023-01-02 23:28:59.430 - [NOTICE] callbacks.on_step_start(182): Cycle 36/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:28:59.476 - [NOTICE] callbacks.on_step_start(182): Cycle 36/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:59.572 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.742 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:28:59.573 - [NOTICE] callbacks.on_cycle_start(174): Cycle 37/300 (8.253 s elapsed) --------------------
2023-01-02 23:28:59.573 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:28:59.625 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 2/4: Rest for 5 min
2023-01-02 23:28:59.638 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:28:59.688 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:28:59.785 - [NOTICE] callbacks.on_cycle_end(196): Capacity

2023-01-02 23:29:02.005 - [NOTICE] callbacks.on_step_start(182): Cycle 48/300, step 2/4: Rest for 5 min
2023-01-02 23:29:02.017 - [NOTICE] callbacks.on_step_start(182): Cycle 48/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:02.067 - [NOTICE] callbacks.on_step_start(182): Cycle 48/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:02.162 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.703 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:02.164 - [NOTICE] callbacks.on_cycle_start(174): Cycle 49/300 (10.844 s elapsed) --------------------
2023-01-02 23:29:02.165 - [NOTICE] callbacks.on_step_start(182): Cycle 49/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:02.211 - [NOTICE] callbacks.on_step_start(182): Cycle 49/300, step 2/4: Rest for 5 min
2023-01-02 23:29:02.222 - [NOTICE] callbacks.on_step_start(182): Cycle 49/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:02.268 - [NOTICE] callbacks.on_step_start(182): Cycle 49/300, s

...

2023-01-02 23:29:43.818 - [NOTICE] callbacks.on_step_start(182): Cycle 247/300, step 2/4: Rest for 5 min
2023-01-02 23:29:43.830 - [NOTICE] callbacks.on_step_start(182): Cycle 247/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:43.868 - [NOTICE] callbacks.on_step_start(182): Cycle 247/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:43.979 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.923 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:43.980 - [NOTICE] callbacks.on_cycle_start(174): Cycle 248/300 (52.660 s elapsed) --------------------
2023-01-02 23:29:43.981 - [NOTICE] callbacks.on_step_start(182): Cycle 248/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:44.023 - [NOTICE] callbacks.on_step_start(182): Cycle 248/300, step 2/4: Rest for 5 min
2023-01-02 23:29:44.034 - [NOTICE] callbacks.on_step_start(182): Cycle 248/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:44.071 - [NOTICE] callbacks.on_step_start(182): Cycle 24

2023-01-02 23:29:46.199 - [NOTICE] callbacks.on_cycle_start(174): Cycle 259/300 (54.879 s elapsed) --------------------
2023-01-02 23:29:46.200 - [NOTICE] callbacks.on_step_start(182): Cycle 259/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:46.243 - [NOTICE] callbacks.on_step_start(182): Cycle 259/300, step 2/4: Rest for 5 min
2023-01-02 23:29:46.254 - [NOTICE] callbacks.on_step_start(182): Cycle 259/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:46.290 - [NOTICE] callbacks.on_step_start(182): Cycle 259/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:46.394 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.861 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:46.395 - [NOTICE] callbacks.on_cycle_start(174): Cycle 260/300 (55.074 s elapsed) --------------------
2023-01-02 23:29:46.395 - [NOTICE] callbacks.on_step_start(182): Cycle 260/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:46.436 - [NOTICE] callbacks.on_step_star

2023-01-02 23:29:48.469 - [NOTICE] callbacks.on_step_start(182): Cycle 270/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:48.569 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.801 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:48.570 - [NOTICE] callbacks.on_cycle_start(174): Cycle 271/300 (57.250 s elapsed) --------------------
2023-01-02 23:29:48.571 - [NOTICE] callbacks.on_step_start(182): Cycle 271/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:48.616 - [NOTICE] callbacks.on_step_start(182): Cycle 271/300, step 2/4: Rest for 5 min
2023-01-02 23:29:48.627 - [NOTICE] callbacks.on_step_start(182): Cycle 271/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:48.669 - [NOTICE] callbacks.on_step_start(182): Cycle 271/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:48.768 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.796 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:48.769 - [NOTICE] callbacks.o

2023-01-02 23:29:50.808 - [NOTICE] callbacks.on_step_start(182): Cycle 282/300, step 2/4: Rest for 5 min
2023-01-02 23:29:50.819 - [NOTICE] callbacks.on_step_start(182): Cycle 282/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:50.861 - [NOTICE] callbacks.on_step_start(182): Cycle 282/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:50.958 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.734 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:50.959 - [NOTICE] callbacks.on_cycle_start(174): Cycle 283/300 (59.639 s elapsed) --------------------
2023-01-02 23:29:50.960 - [NOTICE] callbacks.on_step_start(182): Cycle 283/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:51.007 - [NOTICE] callbacks.on_step_start(182): Cycle 283/300, step 2/4: Rest for 5 min
2023-01-02 23:29:51.017 - [NOTICE] callbacks.on_step_start(182): Cycle 283/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:51.062 - [NOTICE] callbacks.on_step_start(182): Cycle 28

2023-01-02 23:29:53.103 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.669 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:53.105 - [NOTICE] callbacks.on_cycle_start(174): Cycle 294/300 (1 minute, 2 seconds elapsed) --------------------
2023-01-02 23:29:53.105 - [NOTICE] callbacks.on_step_start(182): Cycle 294/300, step 1/4: Discharge at 1.5C until 3V
2023-01-02 23:29:53.168 - [NOTICE] callbacks.on_step_start(182): Cycle 294/300, step 2/4: Rest for 5 min
2023-01-02 23:29:53.179 - [NOTICE] callbacks.on_step_start(182): Cycle 294/300, step 3/4: Charge at 1.5C until 4.2V
2023-01-02 23:29:53.214 - [NOTICE] callbacks.on_step_start(182): Cycle 294/300, step 4/4: Hold at 4.2V until C/50
2023-01-02 23:29:53.314 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.663 Ah (originally 4.849 Ah, will stop at 2.425 Ah)
2023-01-02 23:29:53.315 - [NOTICE] callbacks.on_cycle_start(174): Cycle 295/300 (1 minute, 2 seconds elapsed) --------------------
2023-01-02 23:29:5

In [12]:
qwrqwf

NameError: name 'qwrqwf' is not defined

In [ ]:
pybamm.plot_summary_variables(sol_long)

In [10]:
spm.variables.search("loss")

Loss of active material in negative electrode [%]
Loss of active material in positive electrode [%]
Loss of capacity to SEI [A.h]
Loss of capacity to SEI on cracks [A.h]
Loss of capacity to lithium plating [A.h]
Loss of lithium inventory [%]
Loss of lithium inventory, including electrolyte [%]
Loss of lithium to SEI [mol]
Loss of lithium to SEI on cracks [mol]
Loss of lithium to lithium plating [mol]
Negative electrode ohmic losses
Negative electrode ohmic losses [V]
Positive electrode ohmic losses
Positive electrode ohmic losses [V]
X-averaged battery electrolyte ohmic losses [V]
X-averaged battery solid phase ohmic losses [V]
X-averaged electrolyte ohmic losses
X-averaged electrolyte ohmic losses [V]
X-averaged negative electrode ohmic losses
X-averaged negative electrode ohmic losses [V]
X-averaged positive electrode ohmic losses
X-averaged positive electrode ohmic losses [V]
X-averaged solid phase ohmic losses
X-averaged solid phase ohmic losses [V]


In [13]:
sim_long.plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#         "Electrolyte concentration [mol.m-3]",
#         "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
#         "Negative electrode potential [V]",
#         "Electrolyte potential [V]",
#         "Positive electrode potential [V]",
        "Terminal voltage [V]",
#         "X-averaged cell temperature",
        "Negative electrode SOC",
        "Positive electrode SOC",
#         "SEI thickness [m]",
        "Loss of capacity to SEI [A.h]",
        "Negative electrode capacity [A.h]",
        "Positive electrode capacity [A.h]",
        "Discharge capacity [A.h]",
        "X-averaged negative electrode active material volume fraction",
        "X-averaged positive electrode active material volume fraction",

        "Loss of capacity to SEI on cracks [A.h]"

        
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=440.00541261407807, step=4.40005412614078), …

In [ ]:
#print(list(sol_long.summary_variables.keys()))

In [ ]:
type(sol_long.summary_variables[list(sol_long.summary_variables.keys())[1]])

In [ ]:
# mdic={}
# for i in range (1):
# # range(len(sol_long.summary_variables)):
# #     stringing=''.join([list(sol_long.summary_variables.keys())[i]])
#     mdic={f"{i}": sol_long.summary_variables[list(sol_long.summary_variables.keys())[i]]}
# savemat("summary",mdic)


In [14]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries
Epsilon_n_s_cycled=sol_long["X-averaged negative electrode active material volume fraction"].entries
Epsilon_p_s_cycled=sol_long["X-averaged positive electrode active material volume fraction"].entries

ECM_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]

ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']
cycle_time=sol_long.summary_variables['Time [s]']
# exp = 30e6*sol_0["Cell thickness change [m]"].entries
# exp_n = 30e6*sol_0["Negative Electrode thickness change [m]"].entries
# exp_p = 30e6*sol_0["Positive Electrode thickness change [m]"].entries
# ee_p = sol_0["X-averaged positive electrode porosity"].entries
# ee_n = sol_0["X-averaged negative electrode porosity"].entries
# ee_s = sol_0["X-averaged separator porosity"].entries
# j_n = sol_0["X-averaged negative electrode total interfacial current density [A.m-2]"].entries
# j_p = sol_0["X-averaged positive electrode total interfacial current density [A.m-2]"].entries

# x = sol_0["x [m]"].entries
# c_e_0 = sol_0["Electrolyte concentration [mol.m-3]"].entries

In [15]:
ident=""
mdic = {
    "x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,    
#         "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
#         "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time 
      "Epsilon_n_s_cycled"+ident:Epsilon_n_s_cycled,
        "Epsilon_p_s_cycled"+ident:Epsilon_p_s_cycled
       }
savemat("cell04_activeratio.mat",mdic)

In [ ]:
# savemat("x100_test", mdic)


In [ ]:
# print(x100)

In [ ]:
# import csv
# with open(r"F:\Git Hub rep\CRC\neew\PyBaMM\Untitled Folder\whatever.csv", 'w') as f:
#     # create the csv writer
#     writer = csv.writer(f)

# write a row to the csv file
# i=1
# for lines in sol_long.all_summary_variables:
# #     writer.writerow(words)
#     i=i+1
#     savemat(f'{i:02d}', lines)


In [ ]:
type(x100)

In [ ]:
sol_long.variables.search("charge")

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Current [A]"].entries[1:15133])

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Terminal voltage [V]"].entries[1:15133])

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long["Time [s]"].entries

In [ ]:
sim_long.plot(["Local ECM resistance",
"Local ECM resistance [Ohm]",
               "Resistance [Ohm]"])


In [ ]:
 plt.plot(cycle_num,ECM_res)

In [ ]:
sol_long.summary_variables 

In [ ]:
spm.variables.search("resistance")

In [ ]:
spm.param.n.prim